In [134]:
import os
import pandas as pd

In [135]:
# move all the files in initial_upload_raw_data folder to raw_data

data_directory = os.path.join(os.getcwd(), "raw_data")

# Transform Data

## Main Entities

### Game Schema

In [136]:
# To put game list and game details' data together
df_game = pd.read_csv(os.path.join(data_directory, "game_data.csv"))
df_game_details_data = pd.read_csv(os.path.join(data_directory, "game_details_data.csv"), low_memory=False)

In [137]:
# Remove columns in df_game from df_game_details_data
df_game_details_data_subset = df_game_details_data[[col for col in df_game_details_data.columns if col not in df_game.columns.tolist()] + ["id"]].copy()
df_game_output = df_game.merge(df_game_details_data_subset, on=["id"], how="left")
df_game_output.drop_duplicates(subset=["id"], inplace=True)

In [138]:
#### Add GAME STATUS data into game table

df_status = pd.read_csv(os.path.join(data_directory, "game_status.csv"))
df_game_output = df_game_output.merge(df_status.rename(columns={"game_id": "id"}), on=["id"], how="left")

In [139]:
#### Add ESRB data into game table

df_game_esrb = pd.read_csv(os.path.join(data_directory, "game_esrb.csv"))
df_game_output = df_game_output.merge(df_game_esrb[["name", "game_id"]].rename(columns={"name": "esrb", "game_id": "id"}), how="left", on="id")

In [140]:
#### Remove these columns

col_to_delete = [
    "tba",
    "background_image",
    "rating_top",
    "ratings_count",
    "clip",
    "user_game",
    "saturated_color",
    "dominant_color",
    "community_rating",
    "name_original",
    "description",
    "background_image_additional",
    "reddit_logo",
    "reddit_description",
    "metacritic_url"
]

df_game_output.drop(columns=col_to_delete, inplace=True)

In [141]:
df_game_output.shape

(66301, 35)

In [142]:
df_game_output.iloc[0:5, 0:10]

,slug,name,playtime,released,rating,reviews_text_count,added,metacritic,suggestions_count,updated
0,god-of-war-2,God of War (2018),10,2018-04-20,4.59,62,12079,94.0,580,2023-03-09T23:02:50
1,detroit-become-human,Detroit: Become Human,9,2018-05-25,4.29,43,8768,79.0,463,2023-03-08T08:50:56
2,monster-hunter-world-2,Monster Hunter: World,15,2018-01-26,4.02,17,7610,89.0,571,2023-03-09T21:25:35
3,far-cry-5,Far Cry 5,18,2018-03-27,3.71,22,7180,80.0,374,2023-03-08T08:51:16
4,warhammer-vermintide-2,Warhammer: Vermintide 2,7,2018-03-08,3.66,6,6818,82.0,246,2023-03-07T20:57:23


In [143]:
df_game_output.iloc[0:5, 11:20]

,score,reviews_count,website,screenshots_count,movies_count,creators_count,achievements_count,parent_achievements_count,reddit_url
0,NaN,4340,https://godofwar.playstation.com/,11.0,0.0,13.0,184.0,23.0,https://www.reddit.com/r/GodofWar/
1,NaN,2844,https://www.quanticdream.com/en/detroit-become...,18.0,0.0,5.0,15.0,2.0,https://www.reddit.com/r/DetroitBecomeHuman/
2,NaN,1511,http://www.monsterhunterworld.com,10.0,0.0,11.0,315.0,67.0,https://www.reddit.com/r/MonsterHunterWorld/
3,NaN,1893,NaN,11.0,1.0,8.0,304.0,59.0,https://www.reddit.com/r/farcry/
4,NaN,664,NaN,5.0,1.0,10.0,387.0,102.0,NaN


In [144]:
df_game_output.iloc[0:5, 21:]

,reddit_count,twitch_count,youtube_count,parents_count,additions_count,game_series_count,description_raw,yet,owned,beaten,toplay,dropped,playing,esrb
0,10295.0,0.0,1000000.0,0.0,1.0,11.0,It is a new beginning for Kratos. Living as a ...,618.0,6208.0,3505.0,1074.0,269.0,405.0,NaN
1,9554.0,157.0,1000000.0,0.0,1.0,0.0,"In the future world, androids do almost everyt...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11648.0,112.0,1000000.0,0.0,1.0,14.0,Monster Hunter: World is the fifth game in the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11385.0,131.0,1000000.0,0.0,4.0,7.0,Far Cry 5 is an open-world adventure first-per...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,0.0,367244.0,0.0,1.0,1.0,Warhammer: Vermintide 2 is a first-person acti...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Parent Platform Schema

In [145]:
df_parent_platform = pd.read_csv(os.path.join(data_directory, "parent_platform_data.csv"))
df_parent_platform

,id,name,slug
0,1,PC,pc
1,2,PlayStation,playstation
2,3,Xbox,xbox
3,4,iOS,ios
4,8,Android,android
5,5,Apple Macintosh,mac
6,6,Linux,linux
7,7,Nintendo,nintendo
8,9,Atari,atari
9,10,Commodore / Amiga,commodore-amiga


### Platform Schema

In [146]:
df_platform = pd.read_csv(os.path.join(data_directory, "platform_data.csv"))
df_platform = df_platform[["id", "name", "slug"]].copy()
df_platform

,id,name,slug
0,4,PC,pc
1,187,PlayStation 5,playstation5
2,18,PlayStation 4,playstation4
3,1,Xbox One,xbox-one
4,186,Xbox Series S/X,xbox-series-x
5,7,Nintendo Switch,nintendo-switch
6,3,iOS,ios
7,21,Android,android
8,8,Nintendo 3DS,nintendo-3ds
9,9,Nintendo DS,nintendo-ds


In [147]:
#### Add Parent Platform FK in

df_parent_platform_platform = pd.read_csv(os.path.join(data_directory, "parent_platform_platform.csv"))
df_platform_output = df_platform.merge(df_parent_platform_platform[["platform_id", "parent_platform_id"]].rename(columns={"platform_id": "id"}), how="left", on="id")
df_platform_output

,id,name,slug,parent_platform_id
0,4,PC,pc,1
1,187,PlayStation 5,playstation5,2
2,18,PlayStation 4,playstation4,2
3,1,Xbox One,xbox-one,3
4,186,Xbox Series S/X,xbox-series-x,3
5,7,Nintendo Switch,nintendo-switch,7
6,3,iOS,ios,4
7,21,Android,android,8
8,8,Nintendo 3DS,nintendo-3ds,7
9,9,Nintendo DS,nintendo-ds,7


In [148]:
# CHECK FOR PARENT PLATFORM NOT IN PARENT_PLATFORM_SCHEMA

df_platform_output[~df_platform_output["parent_platform_id"].isin(df_parent_platform["id"])]

,id,name,slug,parent_platform_id


### Publisher Schema

In [149]:
df_publisher = pd.read_csv(os.path.join(data_directory, "publisher_data.csv"))
df_publisher.columns

Index(['id', 'name', 'slug', 'games_count', 'image_background', 'description'], dtype='object')

In [150]:
# Exclude these columns first

df_publisher.drop(columns=["games_count", "image_background"], inplace=True)

In [151]:
df_publisher.sample(5)

,id,name,slug,description
9569,4109,SmashGames,smashgames,NaN
31569,38840,hcgame,hcgame,NaN
27879,41687,Bouncyrock Entertainment,bouncyrock-entertainment,NaN
11701,38520,Crunchyroll Games,crunchyroll-games-llc,NaN
25796,41131,Vladimir Chernenko,vladimir-chernenko,NaN


### Tag Schema

In [152]:
df_tag = pd.read_csv(os.path.join(data_directory, "tag_data.csv"))
df_tag.columns

Index(['id', 'name', 'slug'], dtype='object')

In [153]:
# tag/list API return duplicates

df_tag.drop_duplicates(subset=["id"], inplace=True)

###  Genre Schema

In [154]:
df_genre = pd.read_csv(os.path.join(data_directory, "genre_data.csv"))
df_genre.columns

Index(['id', 'name', 'slug'], dtype='object')

In [155]:
df_genre

,id,name,slug
0,4,Action,action
1,51,Indie,indie
2,3,Adventure,adventure
3,5,RPG,role-playing-games-rpg
4,10,Strategy,strategy
5,2,Shooter,shooter
6,40,Casual,casual
7,14,Simulation,simulation
8,7,Puzzle,puzzle
9,11,Arcade,arcade


### Store Schema

In [156]:
df_store = pd.read_csv(os.path.join(data_directory, "store_data.csv"))
df_store.columns

Index(['id', 'name', 'domain', 'slug'], dtype='object')

In [157]:
df_store

,id,name,domain,slug
0,1,Steam,store.steampowered.com,steam
1,3,PlayStation Store,store.playstation.com,playstation-store
2,2,Xbox Store,microsoft.com,xbox-store
3,4,App Store,apps.apple.com,apple-appstore
4,5,GOG,gog.com,gog
5,6,Nintendo Store,nintendo.com,nintendo
6,7,Xbox 360 Store,marketplace.xbox.com,xbox360
7,8,Google Play,play.google.com,google-play
8,9,itch.io,itch.io,itch
9,11,Epic Games,epicgames.com,epic-games


## Weak Schema

###  Ratings Schema

In [158]:
df_game_ratings = pd.read_csv(os.path.join(data_directory, "game_rating.csv"))
df_game_ratings

,id,title,count,percent,game_id
0,5.0,exceptional,3147.0,72.51,58175
1,4.0,recommended,863.0,19.88,58175
2,3.0,meh,194.0,4.47,58175
3,1.0,skip,136.0,3.13,58175
4,4.0,recommended,133.0,60.18,44641
...,...,...,...,...,...
552,3.0,meh,1.0,100.00,816391
553,1.0,skip,48.0,64.00,452636
554,5.0,exceptional,11.0,14.67,452636
555,3.0,meh,10.0,13.33,452636


In [159]:
df_ratings = df_game_ratings[["id", "title"]].drop_duplicates()
df_ratings

,id,title
0,5.0,exceptional
1,4.0,recommended
2,3.0,meh
3,1.0,skip


## Relationship Schema

###  Game-Platform Schema

In [160]:
df_game_platforms = pd.read_csv(os.path.join(data_directory, "game_platform.csv"))

In [161]:
# no idea why this has so many duplicates
df_game_metacritic = pd.read_csv(os.path.join(data_directory, "game_details_metacritic.csv"))

In [162]:
# Add metacritic score info into this relationship table

df_game_platform = pd.merge(df_game_platforms, df_game_metacritic[["metascore", "url", "platform_id", "game_id"]], how="left", on=["platform_id", "game_id"])
df_game_platform.rename(columns={"url": "metacritic_url"}, inplace=True)
df_game_platform.drop_duplicates(inplace=True)
df_game_platform

,game_id,platform_id,metascore,metacritic_url
0,58175,4,93.0,https://www.metacritic.com/game/pc/god-of-war
3,58175,18,94.0,https://www.metacritic.com/game/playstation-4/...
6,44641,4,NaN,NaN
7,44641,1,NaN,NaN
8,44641,18,NaN,NaN
...,...,...,...,...
2225,917815,4,NaN,NaN
2226,904193,4,NaN,NaN
2227,909793,4,NaN,NaN
2228,913217,4,NaN,NaN


In [163]:
# CHECK FOR PLATFORMS NOT IN PLATFORM_SCHEMA

df_game_platform[~df_game_platform["platform_id"].isin(df_platform["id"])]

,game_id,platform_id,metascore,metacritic_url


###  Game-Ratings Schema

In [164]:
df_game_ratings = df_game_ratings[["id", "count", "game_id"]]
df_game_rating_output = df_game_ratings.rename(columns={"id": "rating_id"})
df_game_rating_output["rating_id"] = df_game_rating_output["rating_id"].astype(int)
df_game_rating_output

,rating_id,count,game_id
0,5,3147.0,58175
1,4,863.0,58175
2,3,194.0,58175
3,1,136.0,58175
4,4,133.0,44641
...,...,...,...
552,3,1.0,816391
553,1,48.0,452636
554,5,11.0,452636
555,3,10.0,452636


### Game-Genre Schema

In [165]:
df_game_genre = pd.read_csv(os.path.join(data_directory, "game_genre.csv"))

In [166]:
df_game_genre["genre_id"] = df_game_genre["genre_id"].astype(int)
df_game_genre_output = df_game_genre[["genre_id", "game_id"]]
df_game_genre_output

,genre_id,game_id
0,3,58175
1,4,58175
2,5,58175
3,10,44641
4,51,44641
...,...,...
3767,1,916515
3768,40,916515
3769,14,916515
3770,15,916515


In [167]:
# No of duplicated row
sum(df_game_genre_output.duplicated())

0

In [168]:
# CHECK FOR GENRE NOT IN GENRE_SCHEMA

df_game_genre_output[~df_game_genre_output["genre_id"].isin(df_genre["id"])]

,genre_id,game_id


### Game-Store Schema

In [169]:
df_game_store = pd.read_csv(os.path.join(data_directory, "game_store.csv"))

In [170]:
df_game_store

,game_id,store_id
0,58175,1
1,58175,3
2,58175,11
3,44641,1
4,44641,3
...,...,...
1911,917815,1
1912,904193,1
1913,909793,1
1914,913217,1


In [171]:
# No of duplicated row
sum(df_game_store.duplicated())

0

In [172]:
# CHECK FOR STORES NOT IN STORE_SCHEMA

df_game_store[~df_game_store["store_id"].isin(df_store["id"])]

,game_id,store_id


### Game-Tag Schema

- Need an additional DAG to fetch Tag data not captured in Tag/List API

In [173]:
df_game_tag = pd.read_csv(os.path.join(data_directory, "game_tag.csv"))

In [174]:
df_game_tag

,game_id,tag_id
0,58175,31
1,58175,42396
2,58175,42417
3,58175,42392
4,58175,40847
...,...,...
25938,916515,66540
25939,916515,49951
25940,916515,58267
25941,916515,82518


In [175]:
# No of duplicated row
sum(df_game_tag.duplicated())

0

In [176]:
# CHECK FOR TAGS NOT IN TAG_SCHEMA

df_game_tag[~df_game_tag["tag_id"].isin(df_tag["id"])]

,game_id,tag_id


### Game-Publisher Schema

In [177]:
df_game_publisher = pd.read_csv(os.path.join(data_directory, "game_details_publisher.csv"))

In [178]:
df_game_publisher["publisher_id"] = df_game_publisher["publisher_id"].astype(int)

In [179]:
df_game_publisher

,publisher_id,game_id
0,11687,58175
1,55134,58175
2,10212,29177
3,10302,29177
4,2150,46889
...,...,...
67541,29358,410877
67542,30100,407595
67543,30813,407599
67544,40637,409601


In [180]:
# Check for duplicates
df_game_publisher[df_game_publisher.duplicated()]

,publisher_id,game_id
41599,3408,630676


In [181]:
# Drop duplicates
df_game_publisher.drop_duplicates(inplace=True)

In [182]:
# CHECK FOR PUBLISHERS NOT IN PUBLISHER_SCHEMA
df_game_publisher[~df_game_publisher["publisher_id"].isin(df_publisher["id"])]

,publisher_id,game_id


# Export Schema Data

In [183]:
data_upload_directory = os.path.join(os.getcwd(), "transformed_data")

In [184]:
# Entity Tables

df_game_output.to_csv(os.path.join(data_upload_directory, "entity_game.csv"), index=False)
df_parent_platform.to_csv(os.path.join(data_upload_directory, "entity_parent_platform.csv"), index=False)
df_platform_output.to_csv(os.path.join(data_upload_directory, "entity_platform.csv"), index=False)
df_publisher.to_csv(os.path.join(data_upload_directory, "entity_publisher.csv"), index=False)
df_tag.to_csv(os.path.join(data_upload_directory, "entity_tag.csv"), index=False)
df_genre.to_csv(os.path.join(data_upload_directory, "entity_genre.csv"), index=False)
df_store.to_csv(os.path.join(data_upload_directory, "entity_store.csv"), index=False)
df_ratings.to_csv(os.path.join(data_upload_directory, "entity_rating.csv"), index=False)

# Relationship Tables
df_game_platform.to_csv(os.path.join(data_upload_directory, "rs_game_platform.csv"), index=False)
df_game_rating_output.to_csv(os.path.join(data_upload_directory, "rs_game_rating.csv"), index=False)
df_game_genre_output.to_csv(os.path.join(data_upload_directory, "rs_game_genre.csv"), index=False)
df_game_store.to_csv(os.path.join(data_upload_directory, "rs_game_store.csv"), index=False)
df_game_tag.to_csv(os.path.join(data_upload_directory, "rs_game_tag.csv"), index=False)

In [185]:
df_game_publisher.to_csv(os.path.join(data_upload_directory, "rs_game_publisher.csv"), index=False)